<a href="https://colab.research.google.com/github/mgfrantz/CalTech-CTME-AramCo-2025/blob/main/notebooks/03_serve_adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!uv pip install -qqq vllm litellm bitsandbytes

In [4]:
import os
from google.colab import userdata
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

Open a terminal 👇 and run th following commands:

1. Add your `HF_TOKEN` env variable (copy from secrets manager 👈)

```sh
export HF_TOKEN=<your_hf_token>
```

2. Serve the model with your adapter

```sh
vllm serve NousResearch/Llama-3.2-1B \
    --max-model-len 2048 \
    --enable-lora \
    --max-lora-rank 32 \
    --lora-modules sql-lora=mgfrantz/NousResearch-Llama-3.2-1B-ctme-sql-demo
```

In [3]:
import os

# Define file URLs and local filenames
urls = [
    "https://raw.githubusercontent.com/mgfrantz/CalTech-CTME-AramCo-2025/refs/heads/main/notebooks/chatml_evaluation_data.jsonl",
    "https://raw.githubusercontent.com/mgfrantz/CalTech-CTME-AramCo-2025/refs/heads/main/notebooks/chatml_training_data.jsonl",
]
filenames = [
    "chatml_evaluation_data.jsonl",
    "chatml_training_data.jsonl",
]

# Download files if they don't exist
for url, filename in zip(urls, filenames):
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        !wget {url} -O {filename}
    else:
        print(f"{filename} already exists.")

--2025-06-25 21:54:19--  https://raw.githubusercontent.com/mgfrantz/CalTech-CTME-AramCo-2025/refs/heads/main/notebooks/chatml_evaluation_data.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34558 (34K) [text/plain]
Saving to: ‘chatml_evaluation_data.jsonl’

chatml_evaluation_d 100%[===================>]  33.75K  --.-KB/s    in 0.007s  

2025-06-25 21:54:20 (4.79 MB/s) - ‘chatml_evaluation_data.jsonl’ saved [34558/34558]

--2025-06-25 21:54:20--  https://raw.githubusercontent.com/mgfrantz/CalTech-CTME-AramCo-2025/refs/heads/main/notebooks/chatml_training_data.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|1

In [27]:
from litellm import completion
from rich import print
import json

In [10]:
eval_conversations = []
with open('chatml_evaluation_data.jsonl', 'r') as f:
    for line in f:
        eval_conversations.append(json.loads(line))

In [13]:
print(eval_conversations[0].get('conversations'))

[{'role': 'system', 'content': 'You are an expert SQL developer specializing in SQLite. Generate accurate SQL queries that follow these requirements:\n\nSQLITE REQUIREMENTS:\n1. Use only standard SQLite-compatible SQL syntax\n2. Avoid PostgreSQL-specific functions like INTERVAL - use date arithmetic instead\n3. Use SQLite date functions: date(), datetime(), julianday() for date calculations\n4. For "recent month" use: WHERE date_column >= date((SELECT MAX(date_column) FROM table_name), \'-1 month\')\n5. For "past 30 days" use: WHERE date_column >= date((SELECT MAX(date_column) FROM table_name), \'-30 days\')\n6. For rolling averages, use window functions or subqueries\n7. Use MAX(appropriate_date_column) to find the most recent date\n8. Use proper SQLite column types: INTEGER, TEXT, REAL, BLOB\n9. Handle foreign key relationships correctly with proper JOIN syntax\n\nQUERY STANDARDS:\n- Write clear, efficient queries\n- Use appropriate aggregation functions (COUNT, SUM, AVG, MAX, MIN)\n

In [28]:
convo = eval_conversations[0].get('conversations')
messages = convo[:-1]
answer = convo[-1]

print("Messages:")
print(messages)

print("\n\nExpected answer:")
print(answer.get('content'))

Messages:

[
    {
        'role': 'system',
        'content': 'You are an expert SQL developer specializing in SQLite. Generate accurate SQL queries that 
follow these requirements:\n\nSQLITE REQUIREMENTS:\n1. Use only standard SQLite-compatible SQL syntax\n2. Avoid 
PostgreSQL-specific functions like INTERVAL - use date arithmetic instead\n3. Use SQLite date functions: date(), 
datetime(), julianday() for date calculations\n4. For "recent month" use: WHERE date_column >= date((SELECT 
MAX(date_column) FROM table_name), \'-1 month\')\n5. For "past 30 days" use: WHERE date_column >= date((SELECT 
MAX(date_column) FROM table_name), \'-30 days\')\n6. For rolling averages, use window functions or subqueries\n7. 
Use MAX(appropriate_date_column) to find the most recent date\n8. Use proper SQLite column types: INTEGER, TEXT, 
REAL, BLOB\n9. Handle foreign key relationships correctly with proper JOIN syntax\n\nQUERY STANDARDS:\n- Write 
clear, efficient queries\n- Use appropriate aggregation functions (COUNT, SUM, AVG, MAX, MIN)\n- Include proper 
GROUP BY and ORDER BY clauses when needed\n- Use table aliases for readability in complex queries'
    },
    {
        'role': 'user',
        'content': 'Database Schema:\nTable: accounts\n  - id (BIGINT, PRIMARY KEY, NOT NULL)\n  - account_number 
(VARCHAR, NOT NULL)\n  - balance (DOUBLE, NOT NULL)\n  - account_type (VARCHAR, NOT NULL)\n  - customer_id (BIGINT,
NOT NULL)\n\nTable: customers\n  - id (BIGINT, PRIMARY KEY, NOT NULL)\n  - name (VARCHAR, NOT NULL)\n  - address 
(VARCHAR)\n  - email (VARCHAR, NOT NULL)\n  - phone (VARCHAR)\n\nTable: loans\n  - id (BIGINT, PRIMARY KEY, NOT 
NULL)\n  - loan_number (VARCHAR, NOT NULL)\n  - amount (DOUBLE, NOT NULL)\n  - interest_rate (DOUBLE, NOT NULL)\n  
- status (VARCHAR, NOT NULL)\n  - customer_id (BIGINT, NOT NULL)\n\nTable: transactions\n  - id (BIGINT, PRIMARY 
KEY, NOT NULL)\n  - transaction_type (VARCHAR, NOT NULL)\n  - amount (DOUBLE, NOT NULL)\n  - timestamp (TIMESTAMP, 
NOT NULL)\n  - account_id (BIGINT, NOT NULL)\n\nQuestion: Who are the top 5 customers by the number of transactions
in the most recent month?'
    }
]

Expected answer:

SELECT c.name, COUNT(t.id) AS transaction_count FROM customers c JOIN accounts a ON c.id = a.customer_id JOIN 
transactions t ON a.id = t.account_id WHERE t.timestamp >= date((SELECT MAX(timestamp) FROM transactions), '-1 
month') GROUP BY c.name ORDER BY transaction_count DESC LIMIT 5;

In [24]:
vllm_url = 'http://localhost:8000/v1/'
model = 'openai/sql-lora'

In [25]:
resp = completion(
    model,
    messages,
    base_url=vllm_url,
    api_key='not-needed',
    max_completion_tokens=128
)

In [26]:
print(resp.choices[0].message.content)

Solution:
1. Recommend database structure changes SUMMARY:
The database should be set up with tables that best support the request. The following tables are strategic in supporting this particular use-case; make notes on these tables if any asked for:
- account_type: a VARCHAR containing (LIST OF) different loan types
- account_number: a VARCHAR 7-digit primary key (foreign key to existing accounts table)
- customer_id: a large unique integer.
2. Identify and use pro’s and con’s for each logical container SUMMARY:
• Find appropriate table types for each type of data. FLOAT and DOUBLE tables include image of details such as fairness,
